## Importer le modèle

In [ ]:
from keras.models import load_model
import numpy as np
model = load_model('../modèles_assez_bons/2020.07.06/model3.h5')
# summarize model.
model.summary()

## Importer l'image

In [ ]:
import rasterio
image_path = '../Données/S2A_MSIL2A_20190227T030651_N0211_R075_T48PYU_20190227T083207/S2A_MSIL2A_20190227T030651_N0211_R075_T48PYU_20190227T083207.SAFE/GRANULE/L2A_T48PYU_A019234_20190227T031435/IMG_DATA/R10m/T48PYU_20190227T030651_'

In [ ]:
IMAGE_PIXEL = 15
IMAGE_WIDTH = 732
BAND_WIDTH = 4

In [ ]:
def reshape(image):
    reshaped_image = []
    for i in range(len(image[0])):
        reshaped_row = []
        for j in range(len(image[0][0])):
            reshaped_cell = []
            #rajouter / 65535
            reshaped_cell.append(image[0][i][j] / 65535)
            reshaped_cell.append(image[1][i][j] / 65535)
            reshaped_cell.append(image[2][i][j] / 65535)
            reshaped_cell.append(image[3][i][j] / 65535)
            reshaped_row.append(reshaped_cell)
        reshaped_image.append(reshaped_row)
    return reshaped_image

In [ ]:
def split_band(band):
    """Split une bande de Sentinel-2 en bande plus petite
    2 916 images de 20x20"""
    result = []
    my_band = band.read(1)
    for x in range(IMAGE_WIDTH):
        columns = []
        for y in range(IMAGE_WIDTH):
            lines = []
            for i in range(IMAGE_PIXEL):
                cells = []
                for j in range(IMAGE_PIXEL):
                    cells.append(my_band[i + IMAGE_PIXEL * x][j + IMAGE_PIXEL * y])
                lines.append(cells)
            columns.append(lines)
        result.append(columns)
    return result

## Faire la prédiction

In [ ]:
def predict(image_path, output_path, model, type='rg+ir'):
    """Créée l'image de sortie en noir et blanc"""
    if type not in ['rgb+ir', 'rgb']:
        print("Les seuls types disponibles sont 'rgb' et 'rg+ir'.")
        return
    blue = rasterio.open(image_path+'B02_10m.jp2', driver='JP2OpenJPEG') #blue
    green = rasterio.open(image_path+'B03_10m.jp2', driver='JP2OpenJPEG') #green
    red = rasterio.open(image_path+'B04_10m.jp2', driver='JP2OpenJPEG') #red
    infra_red = rasterio.open(image_path+'B08_10m.jp2', driver='JP2OpenJPEG') #near impact
    smaller_blue = None
    smaller_infra_red = None
    smaller_blue = split_band(blue)
    print("Fin bande bleue")
    smaller_green = split_band(green)
    print("Fin bande verte")
    smaller_red = split_band(red)
    print("Fin bande rouge")
    smaller_infra_red = split_band(infra_red)
    print("Fin bande infra rouge")
    prediction = []
    prediction_len = set()
    for x in range(IMAGE_WIDTH):
        row_prediction = []
        for y in range(IMAGE_WIDTH):
            smaller_image = []
            smaller_image.append(smaller_infra_red[x][y])
            smaller_image.append(smaller_red[x][y])
            smaller_image.append(smaller_green[x][y])
            smaller_image.append(smaller_blue[x][y])
            value = model.predict(np.expand_dims(reshape(smaller_image), axis=0))
            prediction_len.add(value[0][0] * 65535)
            for i in range(IMAGE_PIXEL):
                row_prediction.append(value[0][0] * 65535)
                
        for i in range(IMAGE_PIXEL):
            prediction.append(np.array(row_prediction).astype('uint16'))
    output = rasterio.open(output_path + '_prediction.tiff', 'w', driver='Gtiff', 
                          width=red.width, height=red.height, count=1, crs=red.crs, 
                          transform=red.transform, 
                          dtype=red.dtypes[0])
    output.write(prediction, 1)
    output.close()
    print("Le fichier de prédiction a été créé.")

In [ ]:
predict(image_path, "output", model, type='rgb')